# 数据读取

读取 CSV 格式的数据和概览数据

In [22]:
import pandas as pd
import numpy as np

# 2016 06 10 20:30:00
# custom_date_parser = lambda x: pd.datetime.strptime(x, "%Y %m %d %H:%M:%S")
# custom_date_parser = lambda x: pd.datetime.strptime(x, "%Y")
# df = pd.read_csv('data/rooms.csv', parse_dates=['Text6'], date_parser=custom_date_parser)
df = pd.read_csv('data/rooms.csv')

In [27]:
df.head()

,Field1_text,Field1_links,Textbox_value,Text,Text1,Text2,Text3,Text4,Text5,Text6,Text7,Text8,Text9,Text10
0,合租·朝丰家园3居室-北卧,https://www.ziroom.com/x/716001508.html,1900,9.4㎡,朝北,3室1厅,7/17,有,朝丰家园,2011,塔板结合,独立供暖,30%,2.5
1,合租·金隅康惠园2号院3居室-南卧,https://www.ziroom.com/x/762105899.html,2100,6.2㎡,朝南,3室1厅,7/20,有,金隅康惠园2号院,2012,塔楼,集体供暖,30%,2.5
2,合租·南平里3居室-东卧,https://www.ziroom.com/x/750726441.html,1900,14.5㎡,朝东,3室1厅,5/6,无,南平里,1998,板楼,集体供暖,30%,1.5
3,合租·富力又一城A区3居室-南卧,https://www.ziroom.com/x/723780132.html,2500,17㎡,朝南,3室1厅,1/24,有,富力又一城A区,2009,板楼,独立供暖,35%,2.38
4,合租·康泉小区3居室-北卧,https://www.ziroom.com/x/725884935.html,2100,9.3㎡,朝北,3室1厅,6/28,有,康泉小区,2015,塔板结合,集体供暖,34%,2.38


In [24]:
df.shape

(3540, 14)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3540 entries, 0 to 3539
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Field1_text    3502 non-null   object
 1   Field1_links   3502 non-null   object
 2   Textbox_value  3540 non-null   int64 
 3   Text           3324 non-null   object
 4   Text1          3324 non-null   object
 5   Text2          3324 non-null   object
 6   Text3          3324 non-null   object
 7   Text4          3324 non-null   object
 8   Text5          3324 non-null   object
 9   Text6          3313 non-null   object
 10  Text7          3313 non-null   object
 11  Text8          3313 non-null   object
 12  Text9          3302 non-null   object
 13  Text10         3285 non-null   object
dtypes: int64(1), object(13)
memory usage: 387.3+ KB


# 文本类型抽取

- 更多处理文本例子参考：https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html
- 正则表达式工具：https://regex101.com/

In [14]:
# 去掉 Field1_text 列为 na 的 rows
df = df[df["Field1_text"].notna()]

In [16]:
# split 会生成多个元素，get(0) 代表只想要第1个元素
df["way"] = df["Field1_text"].str.split("·").str.get(0)

# split 会生成多个元素，expand 属性就是让每个元素对应1列 
df[['floor', 'totalfloor']] = df["Text3"].str.split("/", expand=True)

In [ ]:
# regex 属性表明不用正则去匹配替换
df["Text8"] = df["Text8"].str.replace("%", "", regex=False)

In [18]:
# 正则表达式抽取
df["Text"] = df["Text"].str.extract(r'(\d+\.*\d*)')
df["Text"] = pd.to_numeric(df["Text"])

# 加上 ?P<rooms> 代表列名
df[['rooms', 'ting']]  = df["Text2"].str.extract(r'(?P<rooms>\d+).(?P<ting>\d+)')

# 数据聚合（GroupBy）

GroupBy 以后通常需要聚合函数进行处理，函数分为以下3类：

1. basic math：sum, mean, median, minimum, maximum, standard deviation, variance, mean absolute deviation
2. stats functions from scipy or numpy
3. 自定义的聚合函数

In [3]:
# 加载泰坦尼克的数据
dd = pd.read_csv('data/titanic.csv')

In [8]:
# PassengerId：乘客id
# Survived（是否生存）：0 = No, 1 = Yes
# Pclass：票的等级，1 = 1st, 2 = 2nd, 3 = 3rd
# SibSp：船上 配偶/兄弟 的数量
# Parch：船上 父母/孩子 的数量
# ticket：票号
# fare：票价
# cabin：客舱号
# Embarked：上船的港口，C = Cherbourg, Q = Queenstown, S = Southampton
dd.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## basic math

除了上面提到的 sum，median 等，这个小节还包含 counting：

- count：组内元素的数量（不包含 NaN）
- size：组内元素的数量（包含 NaN）
- nunique：组内不同元素的数量，比如组内有3个数字（3，3，1），那么不同元素的数量为2。dropna 参数决定是否包含 NaN

In [25]:
agg_func_math = {
    'Fare':
    ['sum', 'mean', 'median', 'min', 'max', 'std', 'var', 'mad']
}
dd.groupby(['Embarked']).agg(agg_func_math)

Fare                                                 \
                 sum       mean median     min       max        std   
Embarked                                                              
C         10072.2962  59.954144  29.70  4.0125  512.3292  83.912994   
Q          1022.2543  13.276030   7.75  6.7500   90.0000  14.188047   
S         17439.3988  27.079812  13.00  0.0000  263.0000  35.887993   

                                  
                  var        mad  
Embarked                          
C         7041.390607  53.024568  
Q          201.300677   7.871543  
S         1287.948067  21.299550

In [27]:
agg_func_describe = {'Fare': ['describe']}
# 四舍五入，保留2位小数
dd.groupby(['Embarked']).agg(agg_func_describe).round(2)

Fare                                                
         describe                                                
            count   mean    std   min    25%    50%   75%     max
Embarked                                                         
C           168.0  59.95  83.91  4.01  13.70  29.70  78.5  512.33
Q            77.0  13.28  14.19  6.75   7.75   7.75  15.5   90.00
S           644.0  27.08  35.89  0.00   8.05  13.00  27.9  263.00

In [29]:
# 这几个函数的具体含义参考上面
agg_func_count = {'Survived': ['count', 'nunique', 'size']}
dd.groupby(['Sex']).agg(agg_func_count)

Survived                  
          count nunique size  sum
Sex                              
female      314       2  314  233
male        577       2  577  109

In [33]:
# 排序，获取组内的第一个元素和最后一个元素。
# 实际上，它的效果是和 max，min 一样的。这里仅仅作为演示
agg_func_selection = {'Fare': ['first', 'last']}
dd.sort_values(by=['Fare'], ascending=False).groupby(['Embarked']).agg(agg_func_selection)

Fare        
             first    last
Embarked                  
C         512.3292  4.0125
Q          90.0000  6.7500
S         263.0000  0.0000

## Scipy 或 Numpy 中的统计函数

我只演示 skew 和 mode。理论上来讲，只要能处理数组，然后返回单个数字的函数都是可以的。

In [10]:
from scipy.stats import skew, mode
# mode 函数：获取出现最多的元素 和 出现的次数
# set 函数：找出所有不同的元素（去重）
# pd.Series.mode：只返回 出现最多的元素，不返回次数
agg_func_stats = {'Fare': [skew, mode, pd.Series.mode, set]}
dd.groupby(['Embarked']).agg(agg_func_stats)

Fare                            \
              skew              mode    mode   
Embarked                                       
C         3.305112  ([7.2292], [15])  7.2292   
Q         4.265111    ([7.75], [30])  7.7500   
S         3.640276    ([8.05], [43])  8.0500   

                                                             
                                                        set  
Embarked                                                     
C         {512.3292, 4.0125, 262.375, 7.225, 7.2292, 7.8...  
Q         {6.75, 7.6292, 7.8292, 7.75, 8.4583, 8.0292, 7...  
S         {0.0, 5.0, 6.975, 7.925, 8.05, 7.25, 7.8542, 1...

## 自定义函数

https://pbpython.com/groupby-agg.html

In [36]:
# Define a function
def percentile_25(x):
    return x.quantile(.25)

# Define a lambda function
lambda_25 = lambda x: x.quantile(.25)
lambda_25.__name__ = 'lambda_25%'

agg_func = {
    'Fare': [percentile_25, lambda_25, lambda x: x.quantile(.25)]
}

# 可以看到列名不一样
dd.groupby(['Embarked']).agg(agg_func).round(2)

Fare                      
         percentile_25 lambda_25% <lambda_0>
Embarked                                    
C                13.70      13.70      13.70
Q                 7.75       7.75       7.75
S                 8.05       8.05       8.05

In [15]:
# 跨多列的聚合
def summary(x):
    result = {
        'fare_sum': x['Fare'].sum(),
        'fare_mean': x['Fare'].mean(),
        'fare_range': x['Fare'].max() - x['Fare'].min()
    }
    return pd.Series(result).round(0)

dd.groupby(['Pclass']).apply(summary)


,fare_sum,fare_mean,fare_range
Pclass,,,
1,18177.0,84.0,512.0
2,3802.0,21.0,74.0
3,6715.0,14.0,70.0


In [11]:
dd.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
# assign 函数，分配一个新的 column， https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html
# lambda 里面的变量 x 是整个 dd，也就是 DataFrame 类型
# DataFrame + 1 的计算含义就是每个相关的元素 + 1
dd.groupby(['Embarked', 'Pclass']).agg({
    'Fare': 'sum'
}).round(0).assign(pct_total=lambda x: x + 1)

Fare  pct_total
Embarked Pclass                   
C        1       8901.0     8902.0
         2        431.0      432.0
         3        740.0      741.0
Q        1        180.0      181.0
         2         37.0       38.0
         3        805.0      806.0
S        1       8936.0     8937.0
         2       3334.0     3335.0
         3       5169.0     5170.0

In [13]:
fare_group = dd.groupby(['Embarked', 'Pclass']).agg({'Fare': 'sum'})
fare_group

Fare
Embarked Pclass           
C        1       8901.0750
         2        431.0917
         3        740.1295
Q        1        180.0000
         2         37.0500
         3        805.2043
S        1       8936.3375
         2       3333.7000
         3       5169.3613

In [14]:
# level=0 意味着按照 Embarked 来算累计和（cumulative）
# 8901.0750 + 431.0917 = 9332.1667
fare_group.groupby(level=0).cumsum()

Fare
Embarked Pclass            
C        1        8901.0750
         2        9332.1667
         3       10072.2962
Q        1         180.0000
         2         217.0500
         3        1022.2543
S        1        8936.3375
         2       12270.0375
         3       17439.3988

# Flattening Hierarchical Column Indices

可以看到下面的列名是多层级的，那么如何给它扁平化呢？

In [15]:
dd.groupby(['Embarked', 'Pclass']).agg({'Fare': ['sum', 'mean']}).round(0)

Fare       
                    sum   mean
Embarked Pclass               
C        1       8901.0  105.0
         2        431.0   25.0
         3        740.0   11.0
Q        1        180.0   90.0
         2         37.0   12.0
         3        805.0   11.0
S        1       8936.0   70.0
         2       3334.0   20.0
         3       5169.0   15.0

In [17]:
multi_df = dd.groupby(['Embarked', 'Pclass'],
                    as_index=False).agg({'Fare': ['sum', 'mean']})
multi_df

Embarked Pclass       Fare            
                         sum        mean
0        C      1  8901.0750  104.718529
1        C      2   431.0917   25.358335
2        C      3   740.1295   11.214083
3        Q      1   180.0000   90.000000
4        Q      2    37.0500   12.350000
5        Q      3   805.2043   11.183393
6        S      1  8936.3375   70.364862
7        S      2  3333.7000   20.327439
8        S      3  5169.3613   14.644083

In [18]:
# 修改 column name
multi_df.columns = [
'_'.join(col).rstrip('_') for col in multi_df.columns.values
]
multi_df

,Embarked,Pclass,Fare_sum,Fare_mean
0,C,1,8901.0750,104.718529
1,C,2,431.0917,25.358335
2,C,3,740.1295,11.214083
3,Q,1,180.0000,90.000000
4,Q,2,37.0500,12.350000
5,Q,3,805.2043,11.183393
6,S,1,8936.3375,70.364862
7,S,2,3333.7000,20.327439
8,S,3,5169.3613,14.644083
